In [1]:
# Retrieve packages
from splinter import Browser
from bs4 import BeautifulSoup
import pandas as pd
import time
import re

from sqlalchemy import create_engine, MetaData, Table
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, String, Integer, Float, Date, ForeignKey, PrimaryKeyConstraint
from sqlalchemy.orm import sessionmaker

import sqlite3

1. Scrape website to retrieve Player Data

In [54]:
# Define page

browser = Browser('chrome')

url = "https://basketball.realgm.com/nba/stats"
browser.visit(url)
time.sleep(5)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
browser.quit()


In [55]:
# Retrieve Players
response = soup.find_all(class_="nowrap tablesaw-cell-persist")

players = []
url_player = []

for x in range(len(response)):
    players.append(response[x].text)

    text = response[x]
    href_pattern = r'<a href="([^"]+)">([^<]+)</a>'
    matches = re.findall(href_pattern, str(text))
    href_text = [match[0] for match in matches]
    href_text = href_text[0]
    href_text = "https://basketball.realgm.com/" + href_text
    url_player.append(href_text)



In [57]:
# Find Current Team Function

def find_current_team(soup):
    response = soup.find_all(class_="half-column-left")
     
    current_team_pattern = r'<strong>Current Team:</strong>\s*<a href="[^"]+">([^<]+)</a>'
    match = re.search(current_team_pattern, str(response[0]))
    
    if match:
        current_team = match.group(1)
    else:
        current_team = None
    
    return current_team



In [58]:
# Find age Function

def find_age(soup):
    response = soup.find_all(class_="half-column-left")
        
    age_pattern = r'<strong>Born:</strong>\s*<a href="[^"]+">[^<]+</a>\s*\(([^)]+)\)'
    match = re.search(age_pattern, str(response[0]))
    
    if match:
        age = match.group(1)
    else:
        age = None
    
    return age


In [59]:
# Find height Function

def find_height(soup):
    response = soup.find_all(class_="half-column-left")
        
    height_pattern = r'<strong>Height:</strong>\s*([^<]+)'
    match = re.search(height_pattern, str(response[0]))
    
    if match:
        height = match.group(1).strip()
    else:
        height = None
    
    return height

In [60]:
# Find weight Function

def find_weight(soup):
    response = soup.find_all(class_="half-column-left")
        
    weight_pattern = r'<strong>Weight:</strong>\s*([^<]+)'
    match = re.search(weight_pattern, str(response[0]))
    
    if match:
        weight = match.group(1).strip()
    else:
        weight = None
    
    return weight

In [61]:
# Find current NBA status Function

def find_nba_status(soup):
    response = soup.find_all(class_="half-column-right")
        
    nba_status_pattern = r'<strong>Current NBA Status:</strong>\s*([^<]+)'
    match = re.search(nba_status_pattern, str(response[0]))
    
    if match:
        nba_status = match.group(1).strip()
    else:
        nba_status = None
    
    return nba_status



In [62]:
# Find draft entry Function

def find_draft_entry(soup):
    response = soup.find_all(class_="half-column-right")
        
    draft_entry_pattern = r'<strong>Draft Entry:</strong>\s*<a[^>]+>([^<]+)</a>'
    match = re.search(draft_entry_pattern, str(response[0]))
    
    if match:
        draft_entry = match.group(1).strip()
    else:
        draft_entry = None
    
    return draft_entry

In [50]:
# Retrieve score function

def find_scores(soup):
    response = soup.find_all(class_="per_game")
    
    year_overview = []
    gp_overview = []
    gs_overview = []
    min_overview = []
    pts_overview = []
    fgm_overview = []
    fga_overview = []
    fg_prc_overview = []
    three_pm_overview = []
    three_pa_overview = []
    three_p_prc_overview = []
    ftm_overview = []
    fta_overview = []
    ft_prc_overview = []
    off_overview = []
    deff_overview = []
    trb_overview = []
    ast_overview = []
    stl_overview = []
    blk_overview = []
    tov_overview = []
    pf_overview = []

    for x in range(len(response)-1):
        try:
            overview_scores = response[x+1].find_all('td')
            year = overview_scores[0].text
            year = year[:7]
            gp = overview_scores[2]
            gs = overview_scores[3]
            min = overview_scores[4]
            pts = overview_scores[5]
            fgm = overview_scores[6]
            fga = overview_scores[7]
            fg_prc = overview_scores[8]
            three_pm = overview_scores[9]
            three_pa = overview_scores[10]
            three_p_prc = overview_scores[11]
            ftm = overview_scores[12]
            fta = overview_scores[13]
            ft_prc = overview_scores[14]
            off = overview_scores[15]
            deff = overview_scores[16]
            trb = overview_scores[17]
            ast = overview_scores[18]
            stl = overview_scores[19]
            blk = overview_scores[20]
            tov = overview_scores[21]
            pf = overview_scores[22]

        except:
            pass

        if year[:2] == "20":
            year_overview.append(year)
            gp_overview.append(gp)
            gs_overview.append(gs)
            min_overview.append(min)
            pts_overview.append(pts)
            fgm_overview.append(fgm)
            fga_overview.append(fga)
            fg_prc_overview.append(fg_prc)
            three_pm_overview.append(three_pm)
            three_pa_overview.append(three_pa)
            three_p_prc_overview.append(three_p_prc)
            ftm_overview.append(ftm)
            fta_overview.append(fta)
            ft_prc_overview.append(ft_prc)
            off_overview.append(off)
            deff_overview.append(deff)
            trb_overview.append(trb)
            ast_overview.append(ast)
            stl_overview.append(stl)
            blk_overview.append(blk)
            tov_overview.append(tov)
            pf_overview.append(pf)
            
        else:
            pass
        
    find_scores_df = pd.DataFrame({"Year":year_overview,
                                   "GP":gp_overview,
                                   "GS": gs_overview,
                                   "MIN": min_overview,
                                   "PTS": pts_overview,
                                   "FGM": fgm_overview,
                                   "FGA": fga_overview,
                                   "FG%": fg_prc_overview,
                                   "3PM": three_pm_overview,
                                   "3PA": three_pa_overview,
                                   "3P%": three_p_prc_overview,
                                   "FTM": ftm_overview,
                                   "FTA": fta_overview,
                                   "FT%": ft_prc_overview,
                                   "OFF": off_overview,
                                   "DEF": deff_overview,
                                   "TRB": trb_overview,
                                   "AST": ast_overview,
                                   "STL": stl_overview,
                                   "BLK": blk_overview,
                                   "TOV": tov_overview,
                                   "PF": pf_overview})
    
    
    duplicate_mask = find_scores_df['Year'].duplicated()
    keep_mask = ~duplicate_mask
    find_scores_df = find_scores_df[keep_mask]

    find_scores_df['Year'] = find_scores_df['Year'].astype(str)

    rows_with_value = find_scores_df.index[find_scores_df['Year'] == '2022-23'].tolist()
    find_scores_df = find_scores_df.head(rows_with_value[0]+1)


    return find_scores_df



In [51]:
# Browse to player page and retrieve data

player_overview = []
current_team_overview = []
age_overview = []
height_overview = []
weight_overview =[]
nba_status_overview =[]
draft_entry_overview = []

player_scores_df = pd.DataFrame({"Player":[],
                                 "Year":[],
                                 "GP":[],
                                 "GS": [],
                                 "MIN": [],
                                 "PTS": [],
                                 "FGM": [],
                                 "FGA": [],
                                 "FG%": [],
                                 "3PM": [],
                                 "3PA": [],
                                 "3P%": [],
                                 "FTM": [],
                                 "FTA": [],
                                 "FT%": [],
                                 "OFF": [],
                                 "DEF": [],
                                 "TRB": [],
                                 "AST": [],
                                 "STL": [],
                                 "BLK": [],
                                 "TOV": [],
                                 "PF": []})


for x in range(len(players)):
    browser = Browser('chrome')
    url = url_player[x]
    browser.visit(url)
    time.sleep(5)
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    browser.quit()

    player_overview.append(players[x])
    current_team_overview.append(find_current_team(soup))
    age_overview.append(find_age(soup))
    height_overview.append(find_height(soup))
    weight_overview.append(find_weight(soup))
    nba_status_overview.append(find_nba_status(soup))
    draft_entry_overview.append(find_draft_entry(soup))

    player_temp_df = find_scores(soup)

    player_name_overview = []

    for x in range(len(player_temp_df)):
        player_name_overview.append(players[x])

    player_temp_df["Player"] = player_name_overview
    
    player_scores_df = pd.concat([player_scores_df, player_temp_df], ignore_index=True)
    

# Create Player Info Dataframe

player_info_df = pd.DataFrame({"Player": player_overview,
                               "Current team": current_team_overview,
                               "Age": age_overview,
                               "Height": height_overview,
                               "Weight": weight_overview,
                               "NBA status": nba_status_overview,
                               "Draft entry": draft_entry_overview})
player_info_df.to_csv('players_info.csv', index=False)


In [23]:
# Clean Player Score data

gp_update = []
gs_update = []
min_update = []
pts_update = []
fgm_update = []
fga_update = []
fg_prc_update = []
three_pm_update = []
three_pa_update = []
three_p_prc_update = []
ftm_update = []
fta_update = []
ft_prc_update = []
off_update = []
deff_update = []
trb_update = []
ast_update = []
stl_update = []
blk_update = []
tov_update = []
pf_update = []

for x in range(len(player_scores_df["GP"])):
    gp_value = player_scores_df["GP"]
    gs_value = player_scores_df["GS"]
    min_value = player_scores_df["MIN"]
    pts_value = player_scores_df["PTS"]
    fgm_value = player_scores_df["FGM"]
    fga_value = player_scores_df["FGA"]
    fg_prc_value = player_scores_df["FG%"]
    three_pm_value = player_scores_df["3PM"]
    three_pa_value = player_scores_df["3PA"]
    three_p_prc_value = player_scores_df["3P%"]
    ftm_value = player_scores_df["FTM"]
    fta_value = player_scores_df["FTA"]
    ft_prc_value = player_scores_df["FT%"]
    off_value = player_scores_df["OFF"]
    deff_value = player_scores_df["DEF"]
    trb_value = player_scores_df["TRB"]
    ast_value = player_scores_df["AST"]
    stl_value = player_scores_df["STL"]
    blk_value = player_scores_df["BLK"]
    tov_value = player_scores_df["TOV"]
    pf_value = player_scores_df["PF"]

    try:
        gp_value = float(gp_value[x].text)
        gs_value = float(gs_value[x].text)
        min_value = float(min_value[x].text)
        pts_value = float(pts_value[x].text)
        fgm_value = float(fgm_value[x].text)
        fga_value = float(fga_value[x].text)
        fg_prc_value = float(fg_prc_value[x].text)
        three_pm_value = float(three_pm_value[x].text)
        three_pa_value = float(three_pa_value[x].text)
        three_p_prc_value = float(three_p_prc_value[x].text)
        ftm_value = float(ftm_value[x].text)
        fta_value = float(fta_value[x].text)
        ft_prc_value = float(ft_prc_value[x].text)
        off_value = float(off_value[x].text)
        deff_value = float(deff_value[x].text)
        trb_value = float(trb_value[x].text)
        ast_value = float(ast_value[x].text)
        stl_value = float(stl_value[x].text)
        blk_value = float(blk_value[x].text)
        tov_value = float(tov_value[x].text)
        pf_value = float(pf_value[x].text)
    except:
        pass
    gp_update.append(gp_value)
    gs_update.append(gs_value)
    min_update.append(min_value)
    pts_update.append(pts_value)
    fgm_update.append(fgm_value)
    fga_update.append(fga_value)
    fg_prc_update.append(fg_prc_value)
    three_pm_update.append(three_pm_value)
    three_pa_update.append(three_pa_value)
    three_p_prc_update.append(three_p_prc_value)
    ftm_update.append(ftm_value)
    fta_update.append(fta_value)
    ft_prc_update.append(ft_prc_value)
    off_update.append(off_value)
    deff_update.append(deff_value)
    trb_update.append(trb_value)
    ast_update.append(ast_value)
    stl_update.append(stl_value)
    blk_update.append(blk_value)
    tov_update.append(tov_value)
    pf_update.append(pf_value)


player_scores_df["GP"] = gp_update
player_scores_df["GS"] = gs_update
player_scores_df["MIN"] = min_update
player_scores_df["PTS"] = pts_update
player_scores_df["FGM"] = fgm_update
player_scores_df["FGA"] = fga_update
player_scores_df["FG%"] = fg_prc_update
player_scores_df["3PM"] = three_pm_update
player_scores_df["3PA"] = three_pa_update
player_scores_df["3P%"] = three_p_prc_update
player_scores_df["FTM"] = ftm_update
player_scores_df["FTA"] = fta_update
player_scores_df["FT%"] = ft_prc_update
player_scores_df["OFF"] = off_update
player_scores_df["DEF"] = deff_update
player_scores_df["TRB"] = trb_update
player_scores_df["AST"] = ast_update
player_scores_df["STL"] = stl_update
player_scores_df["BLK"] = blk_update
player_scores_df["TOV"] = tov_update
player_scores_df["PF"] = pf_update

player_scores_df.to_csv('players_score.csv', index=False)
player_scores_df.head()


,Player,Year,GP,GS,MIN,PTS,FGM,FGA,FG%,3PM,...,FTA,FT%,OFF,DEF,TRB,AST,STL,BLK,TOV,PF
0,Joel Embiid,2016-17,31.0,31.0,25.4,20.23,6.45,13.84,0.466,1.16,...,7.87,0.783,1.97,5.87,7.84,2.13,0.87,2.45,3.77,3.61
1,Luka Doncic,2017-18,63.0,63.0,30.4,22.94,8.1,16.76,0.483,1.05,...,7.41,0.769,2.27,8.68,10.95,3.16,0.63,1.76,3.71,3.32
2,Damian Lillard,2018-19,64.0,64.0,33.7,27.52,9.06,18.73,0.484,1.23,...,10.14,0.804,2.5,11.09,13.59,3.66,0.72,1.91,3.53,3.3
3,Shai Gilgeous-Alexander,2019-20,51.0,51.0,29.5,22.98,7.49,15.71,0.477,1.12,...,8.53,0.807,2.76,8.86,11.63,2.98,0.88,1.27,3.12,3.41
4,Giannis Antetokounmpo,2020-21,51.0,51.0,31.1,28.45,9.04,17.63,0.513,1.14,...,10.75,0.859,2.22,8.35,10.57,2.84,0.98,1.35,3.12,2.41


2. Scrape website to retrieve Team Data

In [27]:
# Define page
browser = Browser('chrome')
url = "https://basketball.realgm.com/nba/team-stats"
browser.visit(url)
time.sleep(5)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
browser.quit()

In [28]:
# Retrieve team & stats

team_response = soup.find_all('tr')

team_overview = []
gp_overview =[]
mpg_overview = []
ppg_overview = []
fgm_overview =[]
fga_overview =[]
fg_prc_overview = []
three_pm_overview =[]
three_pa_overview =[]
three_p_prc = []
ftm_overview =[]
fta_overview =[]
ft_prc_overview =[]
orb_overview = []
drb_overview = []
rpg_overview =[]
apg_overview = []
spg_overview = []
bpg_overview =[]
tov_overview = []
pf_overview =[]


for x in range(1,31):
    response = team_response[x].find_all('td')
    team_overview.append(response[1].text)
    gp_overview.append(response[2].text)
    mpg_overview.append(response[3].text)
    ppg_overview.append(response[4].text)
    fgm_overview.append(response[5].text)
    fga_overview.append(response[6].text)
    fg_prc_overview.append(response[7].text)
    three_pm_overview.append(response[8].text)
    three_pa_overview.append(response[9].text)
    three_p_prc.append(response[10].text)
    ftm_overview.append(response[11].text)
    fta_overview.append(response[12].text)
    ft_prc_overview.append(response[13].text)
    orb_overview.append(response[14].text)
    drb_overview.append(response[15].text)
    rpg_overview.append(response[16].text)
    apg_overview.append(response[17].text)
    spg_overview.append(response[18].text)
    bpg_overview.append(response[19].text)
    tov_overview.append(response[20].text)
    pf_overview.append(response[21].text)

In [30]:
# Create dataframe of results

team_scores_df = pd.DataFrame({"Team": team_overview,
                               "GP": gp_overview,
                               "MPG": mpg_overview,
                               "PPG": ppg_overview,
                               "FMG": fgm_overview,
                               "FGA": fga_overview,
                               "FG%": fg_prc_overview,
                               "3PM": three_pm_overview,
                               "3PA": three_pa_overview,
                               "3P%": three_p_prc,
                               "FTM": ftm_overview,
                               "FTA": fta_overview,
                               "FT%": ft_prc_overview,
                               "ORB": orb_overview,
                               "DRB": drb_overview,
                               "RPG": rpg_overview,
                               "APG": apg_overview,
                               "SPG": spg_overview,
                               "BPG": bpg_overview,
                               "TOV": tov_overview,
                               "PF": pf_overview})
team_scores_df.to_csv('team_score.csv', index=False)
team_scores_df.head()


,Team,GP,MPG,PPG,FMG,FGA,FG%,3PM,3PA,3P%,...,FTA,FT%,ORB,DRB,RPG,APG,SPG,BPG,TOV,PF
0,Sacramento,82,48.4,120.7,43.6,88.2,.494,13.8,37.3,.369,...,25.1,.790,9.5,32.9,42.5,27.3,7.0,3.4,13.5,19.7
1,Golden State,82,48.4,118.9,43.1,90.2,.479,16.6,43.2,.385,...,20.2,.794,10.5,34.1,44.6,29.8,7.2,3.9,16.3,21.4
2,Atlanta,82,48.4,118.4,44.6,92.4,.483,10.8,30.5,.352,...,22.6,.818,11.2,33.2,44.4,25.0,7.1,4.9,12.9,18.8
3,Boston,82,48.7,117.9,42.2,88.8,.475,16.0,42.6,.377,...,21.6,.812,9.7,35.6,45.3,26.7,6.4,5.2,13.4,18.8
4,Oklahoma City,82,48.4,117.5,43.1,92.6,.465,12.1,34.1,.356,...,23.7,.809,11.4,32.3,43.6,24.4,8.2,4.2,13.0,21.0


3. Save data in DB

In [2]:
# Delete Tables

database_url = 'sqlite:///NBA_stats.sqlite'

engine = create_engine(database_url)

Session = sessionmaker(bind=engine)
session = Session()

metadata = MetaData(bind=engine)

table_name_1 = 'Player_info_table'
table_name_2 = 'Player_score_table'
table_name_3 = 'Team_table'

if engine.has_table(table_name_1):
    existing_table = Table(table_name_1, metadata, autoload=True)

    existing_table.drop()
    metadata.remove(existing_table)

    session.commit()

else:
    pass

if engine.has_table(table_name_2):
    existing_table = Table(table_name_2, metadata, autoload=True)

    existing_table.drop()
    metadata.remove(existing_table)

    session.commit()

else:
    pass

if engine.has_table(table_name_3):
    existing_table = Table(table_name_3, metadata, autoload=True)

    existing_table.drop()
    metadata.remove(existing_table)

    session.commit()

else:
    pass

C:\Users\nysbr\AppData\Local\Temp\ipykernel_8660\3695008712.py:16: SADeprecationWarning: The Engine.has_table() method is deprecated and will be removed in a future release.  Please refer to Inspector.has_table(). (deprecated since: 1.4)
  if engine.has_table(table_name_1):
C:\Users\nysbr\AppData\Local\Temp\ipykernel_8660\3695008712.py:27: SADeprecationWarning: The Engine.has_table() method is deprecated and will be removed in a future release.  Please refer to Inspector.has_table(). (deprecated since: 1.4)
  if engine.has_table(table_name_2):
C:\Users\nysbr\AppData\Local\Temp\ipykernel_8660\3695008712.py:38: SADeprecationWarning: The Engine.has_table() method is deprecated and will be removed in a future release.  Please refer to Inspector.has_table(). (deprecated since: 1.4)
  if engine.has_table(table_name_3):


In [3]:
# Create Player Info DB

Base = declarative_base()
class PlayerInfo(Base):
    __tablename__="Player_info_table"
    Player = Column(String(30), primary_key=True)
    Current_team = Column(String(30))
    Age = Column(String(30))
    Height = Column(String(30))	
    Weight = Column(String(30))	
    NBA_status = Column(String(50))
    Draft_entry = Column(String(50))

engine = create_engine("sqlite:///NBA_stats.sqlite")
conn = engine.connect()

Base.metadata.create_all(engine)
session = Session(bind=engine)

session.commit()
session.close()


In [4]:
# Upload Player Info Data

df = pd.read_csv(r'data/players_info.csv')

engine = create_engine('sqlite:///NBA_stats.sqlite')
connection = engine.connect()

table_name = 'Player_info_table'
df.to_sql(table_name, connection, if_exists='append', index=False)

connection.close()

In [5]:
# Create Player Score DB

Base = declarative_base()
class YourTable(Base):
    __tablename__ = 'Player_score_table'
    Player = Column(String(30))
    Year = Column(String(4))
    GP = Column(Float)
    GS = Column(Float)
    MIN = Column(Float)
    PTS = Column(Float)
    FGM = Column(Float)
    FGA = Column(Float)
    FG_prc = Column(Float)
    three_PM = Column(Float)
    Three_PA = Column(Float)
    Three_P_prc	= Column(Float)
    FTM = Column(Float)
    FTA = Column(Float)
    FT_prc = Column(Float)
    OFF	= Column(Float)
    DEF = Column(Float)
    TRB = Column(Float)
    AST = Column(Float)
    STL = Column(Float)
    BLK = Column(Float)
    TOV = Column(Float)
    PF = Column(Float)

    __table_args__ = (PrimaryKeyConstraint('Player', 'Year')),

engine = create_engine("sqlite:///NBA_stats.sqlite")
conn = engine.connect()

Base.metadata.create_all(engine)
session = Session(bind=engine)

session.commit()
session.close()

In [6]:
# Upload Player Score Data

df = pd.read_csv(r'data/players_score.csv')

engine = create_engine('sqlite:///NBA_stats.sqlite')
connection = engine.connect()

table_name = 'Player_score_table'
df.to_sql(table_name, connection, if_exists='append', index=False)

connection.close()

In [7]:
# Create Team DB

Base = declarative_base()
class YourTable(Base):
    __tablename__ = 'Team_table'
    Team = Column(String(50), primary_key=True)
    GP = Column(Float)
    MPG = Column(Float)
    PPG = Column(Float)
    FMG = Column(Float)
    FGA = Column(Float)
    FG_prc = Column(Float)
    Three_PM = Column(Float)
    Three_PA = Column(Float)
    Three_P_prc	= Column(Float)
    FTM = Column(Float)
    FTA = Column(Float)
    FT_prc = Column(Float)
    ORB	= Column(Float)
    DRB = Column(Float)
    RPG = Column(Float)
    APG = Column(Float)
    SPG = Column(Float)
    BPG = Column(Float)
    TOV = Column(Float)
    PF = Column(Float)

engine = create_engine("sqlite:///NBA_stats.sqlite")
conn = engine.connect()

Base.metadata.create_all(engine)
session = Session(bind=engine)

session.commit()
session.close()

In [8]:
# Upload Team Data

df = pd.read_csv(r'data/team_score.csv')

engine = create_engine('sqlite:///NBA_stats.sqlite')
connection = engine.connect()

table_name = 'Team_table'
df.to_sql(table_name, connection, if_exists='append', index=False)

connection.close()